# Schema Registry

## Мотивация

До настоящего момента в топики записывались данные в текстовом формате, но на практике часто приходится работать с объектами определенного формата. Не смотря на то, что любой объект можно преобразовать в json, а потом восстановить его из json строки, накладные расходы на передачу и хранение таких данных могут быть неоправданными.

При работе с Kafka вместо json используют бинарные данные:
- продюсеры отправляют сериализуют объекты в бинарный формат (последовательность 0 и 1);
- консьюмеры читают сообщения из топика в бинарном формате и преобразуют их в обратно в объекты;
- продюсеры и консьюмеры должны использовать одну и ту же схему бинарного представления данных.

На небольших проектах, где продюсеры и консьюмеры находятся под управлением одной команды разработки, договориться о формате не составляет труда. Но когда во взаимодействии участвуют несколько команд, то координация бинарных форматов может стать серьезной проблемой, т.к. необходимо:

- согласовывать формат;
- отслеживать изменения формата;
- обновлять передставление объектов после изменения формата.

[Schema Registry](https://docs.confluent.io/platform/current/schema-registry/index.html) выступает центральным репозиторием схем, что позволяет решить все проблемы, связанные с форматами данных. Schema Regisry - это отдельный сервис, у которого есть REST API интерфейс. Schema Registry поддерживает схемы в форматах:

- AVRO,
- PROTOBUF,
- JSON SCHEMA.

При этом продюсеры и консьюмеры могут генерировать код автоматически на базе файла с форматом.

## Подготовка

Продюсеры и консьюмеры будут использовать топик `my-java-api-topic`:

In [ ]:
kafka-topics --bootstrap-server "${KAFKA_HOST}":"${KAFKA_PORT}" \
    --create \
    --topic my-java-api-topic \
    --partitions 10 \
    --replication-factor 1

## Работа со Schema Registry

### Зарегистрировать локальную схему в Schema Registry

При наличии схемы в проекте, можно опубликовать ее в schema registry. В проекте `kafka-java-demo` находятся 2 схемы:

- схема для ключа в `PROTBUF` формате,
- схема для значения в `AVRO` формате.

На практике лучше выбирать один из форматов.

In [ ]:
execute \
cat kafka-java-demo/src/main/avro/customer-value.avsc | json_pp

In [ ]:
execute \
cat kafka-java-demo/src/main/proto/customer-key.proto

Регистрация схемы возможна:

- при помощи maven плагина [`schema-registry`](https://docs.confluent.io/platform/current/schema-registry/develop/maven-plugin.html),
- через REST API.

#### Регистрация через Maven плагин

При помощи цели [`register`](https://docs.confluent.io/platform/current/schema-registry/develop/maven-plugin.html#schema-registry-register) можно зарегистрировать локальные схемы:

In [ ]:
source ~/.bash_aliases

In [ ]:
mvn schema-registry:register > /tmp/register.log ;
    tail -n 40 /tmp/register.log

In [ ]:
curl -s http://schema-registry:8081/subjects | json_pp

#### Регистрация через REST API

Регистрация схемы равносильна обновлению схемы. При обновлении Schema Registry будет следить за тем, чтобы внесенные изменения не оказывали эффект на имеющихся потребителей схемы.

Например, **нельзя** добавить новую колонку, у которой нет значения по умолчанию:

In [ ]:
cat <<EOF > /tmp/update-customer-value-bad.json
{
    "schema": "{ \
        \"type\": \"record\", \
        \"name\": \"Customer\", \
        \"namespace\": \"com.github.neshkeev.kafka.customer.avro\", \
        \"fields\":[ \
            { \
                \"name\":\"id\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"name\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"age\", \
                \"type\":\"string\" \
            } \
        ] \
    }" \
}
EOF

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions \
    -X POST -H "Content-Type: application/vnd.schemaregistry.v1+json" \
    --data "@/tmp/update-customer-value-bad.json" | json_pp

Обновление схемы не произошло, новая версия схемы `customer-value` не появилась:

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions

Можно убедиться, что в текущей версии отсутствует новая колонка `age`:

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions/latest |
    json_pp |
    sed -n '/schema/{s/^\s\+"schema" : "\(.*\)",/\1/;s,\\",",gp}' |
    json_pp

Для сохранения обратной совместимости при обновлении схемы необходимо **обязательно** нужно указать значение по умолчанию при добавлении новых полей.

Например, можно указать значение `0` в качестве значения по умолчанию для нового поля `age`:

In [ ]:
cat <<EOF >/tmp/update-customer-value-good.json
{
    "schema": "{ \
        \"type\": \"record\", \
        \"name\": \"Customer\", \
        \"namespace\": \"com.github.neshkeev.kafka.customer.avro\", \
        \"fields\":[ \
            { \
                \"name\":\"id\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"name\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"age\", \
                \"type\":\"string\", \
                \"default\":\"0\" \
            } \
        ] \
    }" \
}
EOF

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions \
    -X POST -H "Content-Type: application/vnd.schemaregistry.v1+json" \
    --data "@/tmp/update-customer-value-good.json" | json_pp

Появилась новая версия схемы `customer-value`:

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions

Текущая версия схемы содержит поле `age`, в котором в качестве значения по умолчанию используется `0`:

In [ ]:
curl -s http://schema-registry:8081/subjects/customer-value/versions/latest |
    json_pp |
    sed -n '/schema/{s/^\s\+"schema" : "\(.*\)",/\1/;s,\\",",gp}' |
    json_pp

### Обновить схему в проекте

После внесения изменений в схему, потребители должны обновить файл со схемой в проекте. Цель [`download`](https://docs.confluent.io/platform/current/schema-registry/develop/maven-plugin.html#schema-registry-download) позволяет загрузить актуальную версию схемы из Schema Registry и положить в проект.

Обновить схему можно при помощи maven плагина [`schema-registry`](https://docs.confluent.io/platform/current/schema-registry/develop/maven-plugin.html):

In [ ]:
mvn schema-registry:download > /tmp/schema-download.log ;
    tail -n 40 /tmp/schema-download.log

Новая версия схемы была сохранена на локальный жесткий диск в файлы с проектом:

In [ ]:
execute \
cat kafka-java-demo/src/main/avro/customer-value.avsc | json_pp

### Проверить актуальность схемы

После внесения изменений в схему необходимо убедиться, что обратная совместимость сохраняется.

Цель [`test-compatibility`](https://docs.confluent.io/platform/current/schema-registry/develop/maven-plugin.html#schema-registry-test-compatibility) позволяет проверить соотвествует ли версия схемы в проекте текущей схеме в Schema Registry:

In [ ]:
mvn schema-registry:test-compatibility > /tmp/schema-test-compatibility.log ;
    tail -n 40 /tmp/schema-test-compatibility.log

Если внести несогласованные изменения в схему, то плагин укажет на это. Например, изменение типа поля `id` с `string` на `int` является изменением, которое ломает обратную совместимость:

In [ ]:
execute \
sed -i 's,string,int,' kafka-java-demo/src/main/avro/customer-value.avsc

In [ ]:
execute \
cat kafka-java-demo/src/main/avro/customer-value.avsc | json_pp

Проверка совместимости укажет на нарушение условия обратной совместимости:

In [ ]:
mvn schema-registry:test-compatibility > /tmp/schema-test-compatibility.log ;
    tail -n 40 /tmp/schema-test-compatibility.log

Для отката изменений можно заново скачать актуальную версию:

In [ ]:
mvn schema-registry:download > /tmp/schema-test-compatibility.log ;
    tail -n 40 /tmp/schema-test-compatibility.log

### Генерация java кода

По файлам схемы можно сгенерировать код на любом языке. В java существуют maven плагины для этих целей:

- `protobuf-maven-plugin` генерирует java код на основании файлов в `src/main/proto` директории,
- `avro-maven-plugin` генерирует java код на основании файлов в `src/main/avro` директории.

Код генерируется перед фазой `compile` (компиляция).

Сам сгенерированный код размещается в директории `target/generated-sources` и никогда не должен попадать в git репозиторий:

In [ ]:
mvn compile > /tmp/compile.log ;
    tail -n 20 /tmp/compile.log

In [ ]:
execute \
find kafka-java-demo/target/generated-sources -name "*.java"

Сгенерированные java классы можно использовать как и любые другие классы в java проекте. Например, подключить их в тесты.

Тест `SimpleBinaryTest` использует `CustomerKey.Key` и `Customer` в качестве типовых параметров для `KafkaProducer` и `KafkaConsumer`:

In [ ]:
execute \
sed -n '/class.SimpleBinaryTest/,/KafkaConsumer/p' kafka-java-demo/src/test/java/com/github/neshkeev/kafka/SimpleBinaryTest.java

In [ ]:
mvn test -Dtest=com.github.neshkeev.kafka.SimpleBinaryTest > /tmp/binary-test.log ; 
    tail -n 40 /tmp/binary-test.log

## Задание

1. Загрузить схему `orders` в Schema Registry через REST API:
```bash
cat <<EOF > /tmp/orders-value.json
{
    "schema": "{ \
        \"type\": \"record\", \
        \"name\": \"Orders\", \
        \"namespace\": \"com.github.neshkeev.kafka.customer.avro\", \
        \"fields\":[ \
            { \
                \"name\":\"id\", \
                \"type\":\"string\" \
            }, \
            { \
                \"name\":\"name\", \
                \"type\":\"string\" \
            } \
        ] \
    }" \
}
EOF
```
2. Скачать схему в java проект (maven);
3. Сгенерировать java код;
4. Создать топик orders;
5. Записать сообщение в формате `ORDERS_SCHEMA`;
6. Прочитать сообщение в формате `ORDERS_SCHEMA`;
7. Добавить поле `createdDate` в формате `long` (количество миллисекунд с начала эпохи);
8. Обновить схему в schema-registry;
9. Обновить файл схемы в java проекте;
10. Проверить работоспособность схемы.